# Notebook to query for Fiddler Metrics

In [115]:
import fiddler as fdl

In [116]:
import urllib.parse

In [117]:
import json
import requests
import pandas as pd

In [118]:
from collections import namedtuple

In [119]:
import toml

In [120]:
from copy import deepcopy

In [121]:
from uuid import uuid4

In [122]:
import datetime
from datetime import timedelta

In [123]:
import pandas as pd

In [124]:
parsed_secrets = toml.load('../.streamlit/secrets.toml')

In [125]:
BASE_URL = 'https://preprod.cloud.fiddler.ai'
METRICS_URL = urllib.parse.urljoin(BASE_URL, '/v3/models/{model_id}/metrics')
QUERY_URL = urllib.parse.urljoin(BASE_URL, '/v3/queries')

In [126]:
ID = 'id'
NAME = 'name'
COLUMN_INFO= 'ColumnInfo'

In [127]:
TOKEN = parsed_secrets["FIDDLER_ACCESS_TOKEN"]

In [128]:
GET_HEADERS = {
    'Authorization': f'Bearer {TOKEN}',
}

POST_HEADERS = {
    'Authorization': f'Bearer {TOKEN}',
    'Content-Type': 'application/json',
    'X-Integration': 'FiddleBot'
}

In [129]:
COLUMNS = 'columns'

# Constants for main payload keys
PROJECT_ID = "project_id"
TIME_COMPARISON = "time_comparison"
QUERY_TYPE = "query_type"
FILTERS = "filters"
QUERIES = "queries"

# Constants for filters keys
FILTERS_TIME_LABEL = "time_label"
FILTERS_TIME_RANGE = "time_range"
FILTERS_TIME_ZONE = "time_zone"
FILTERS_BIN_SIZE = "bin_size"

# Constants for time_range keys
TIME_RANGE_START_TIME = "start_time"
TIME_RANGE_END_TIME = "end_time"

# Constants for queries item keys
QUERY_KEY = "query_key"
MODEL_ID = "model_id"
BASELINE_ID = "baseline_id"
METRIC = "metric"
CATEGORIES = "categories"
SEGMENT_ID = "segment_id"
SEGMENT = "segment"
VIZ_TYPE = "viz_type"

# Constants for segment keys
SEGMENT_ID_KEY = "id"
SEGMENT_DEFINITION = "definition"

PREV_DAY = "Previous Day"
PREV_WEEK = "Previous Week"

VIZ_TYPE_LINE = "line"
MONITORING = "MONITORING"

QUERY_PARAM = {
    QUERY_KEY: None,
    MODEL_ID: None,
    BASELINE_ID: None,
    METRIC: None,
    COLUMNS: [],
    CATEGORIES: [],
    SEGMENT: {},
    VIZ_TYPE: None,
}

QUERY_PAYLOAD = {
    PROJECT_ID: None,
    QUERY_TYPE: MONITORING,
    FILTERS: {
        FILTERS_TIME_LABEL: None,
        FILTERS_TIME_RANGE: {
            TIME_RANGE_START_TIME: None,
            TIME_RANGE_END_TIME: None,
        },
        FILTERS_TIME_ZONE: None,
        FILTERS_BIN_SIZE: None,
    },
    QUERIES: [],
}

WEEK = "Week"
DAYS_7 = "7d"
DAY = "Day"
REQUIRES_BASELINE = "requires_baseline"
REQUIRES_CATEGORIES = "requires_categories"

In [130]:
DATA = 'data'
RESULTS = 'results'

In [131]:
DATE = 'date'
VALUE = 'value'

In [132]:
fdl.init(
    url="https://preprod.cloud.fiddler.ai",
    token=TOKEN,
)

250702T12:14:17.281Z     INFO| attached stderr handler to logger: auto_attach_log_handler=True, and root logger not configured 
250702T12:14:17.294Z     INFO| http: https://preprod.cloud.fiddler.ai/v3/server-info GET -- emit req (0 B, timeout: (5, 15)) 
250702T12:14:18.126Z     INFO| http: https://preprod.cloud.fiddler.ai/v3/server-info GET -- resp code: 200, took 0.832 s, resp/req body size: (923 B, 0 B) 
250702T12:14:18.131Z     INFO| http: https://preprod.cloud.fiddler.ai/v3/version-compatibility GET -- emit req (0 B, timeout: (5, 15)) 
250702T12:14:18.392Z     INFO| http: https://preprod.cloud.fiddler.ai/v3/version-compatibility GET -- resp code: 200, took 0.260 s, resp/req body size: (2 B, 0 B) 


In [133]:
PROJECT_NAME = 'py_test'
MODEL_NAME = 'airline_delay'
MODEL_NAME = 'bank_churn_simple_monitoring'

In [134]:
ColumnInfo = namedtuple(COLUMN_INFO, [ID, NAME])

In [135]:
TYPE = 'type'
COLUMNS = 'columns'
PERFORMANCE = 'performance'

In [136]:
PERF_METRIC = 'PerfMetric'
PerfMetricInfo = namedtuple(PERF_METRIC, [ID, NAME])

In [137]:
def get_current_time() -> str:
    """Get the current time"""
    return datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%S")


def get_days_ago(num_days: int) -> str:
    """Convert a number of days to a datetime string"""
    return (datetime.datetime.now() - timedelta(days=num_days)).strftime("%Y-%m-%dT%H:%M:%S")

In [138]:
def get_model(project_name: str, model_name: str):
    """Get a model given project name and model name

    Args:
        project_name: Name of project
        model_name: Name of model
    """
    try:
        project = fdl.Project.from_name(name=project_name)
        model = fdl.Model.from_name(name=model_name, project_id=project.id)
    except Exception as e:
        return "Error in obtaining model"
    return model

In [139]:
def get_model_metrics(project_name: str, model_name: str):
    """Get all metrics associated with the model"""
    project = fdl.Project.from_name(name=project_name)
    model = fdl.Model.from_name(project_id=project.id, name=model_name)
    metrics_url = METRICS_URL.format(model_id=model.id)
    response = requests.get(metrics_url, headers=GET_HEADERS)
    return response.json()

In [140]:
def extract_model_columns(all_metrics: dict):
    '''Extract the name and IDs of all columns in the model'''
    column_details = all_metrics['data']['columns']
    column_id_names = []
    for column in column_details:
        column_id_names.append(ColumnInfo(column[ID], column[NAME]))
    return column_id_names

In [141]:
def extract_performance_metrics(all_metrics: dict):
    """Extract performance metrics associated with the model"""
    raw_details = all_metrics['data']['metrics']
    performance_metrics = []
    for details in raw_details:
        if details['type'] == 'performance':
            performance_metrics.append(PerfMetricInfo(details['id'], details['name']))
    return performance_metrics

In [142]:
{
    "project_id": "0dde103e-bee5-4cbf-b28f-954b99cb6f32",
    "query_type": "MONITORING",
    "filters": {
        "bin_size": "Day",
        "time_zone": "UTC",
        "time_label": "7d",
        "time_range": {
            "start_time": "2025-06-25T00:00:00",
            "end_time": "2025-07-01T23:59:59",
        },
    },
    "queries": [
        {
            "baseline_id": "",
            "columns": [],
            "metric": "f1_score",
            "model_id": "431b97c7-1800-4448-b777-b3c7ba56c4a9",
            "query_key": "1751361504872",
            "segment": {},
            "viz_type": "line",
        }
    ],
}

{'project_id': '0dde103e-bee5-4cbf-b28f-954b99cb6f32',
 'query_type': 'MONITORING',
 'filters': {'bin_size': 'Day',
  'time_zone': 'UTC',
  'time_label': '7d',
  'time_range': {'start_time': '2025-06-25T00:00:00',
   'end_time': '2025-07-01T23:59:59'}},
 'queries': [{'baseline_id': '',
   'columns': [],
   'metric': 'f1_score',
   'model_id': '431b97c7-1800-4448-b777-b3c7ba56c4a9',
   'query_key': '1751361504872',
   'segment': {},
   'viz_type': 'line'}]}

In [143]:
def query_performance_metrics(project_name: str, model_name: str, start_date: str, end_date: str):
    '''Get the performance metrics for a model, given the start and end dates'''
    model = get_model(project_name, model_name)
    all_metrics = get_model_metrics(project_name, model_name)
    performance_metrics = extract_performance_metrics(all_metrics)
    query_filters = []
    for metric in performance_metrics:
        query_filter = deepcopy(QUERY_PARAM)
        query_filter[QUERY_KEY] = str(uuid4())
        query_filter[MODEL_ID] = str(model.id)
        query_filter[BASELINE_ID] = ""
        query_filter[METRIC] = metric.id
        query_filter[VIZ_TYPE] = VIZ_TYPE_LINE
        query_filters.append(query_filter)
    
    payload = deepcopy(QUERY_PAYLOAD)
    payload[PROJECT_ID] = str(model.project_id)
    payload[FILTERS][FILTERS_BIN_SIZE] = DAY
    payload[FILTERS][FILTERS_TIME_LABEL] = DAYS_7
    payload[FILTERS][FILTERS_TIME_RANGE][TIME_RANGE_START_TIME] = start_date
    payload[FILTERS][FILTERS_TIME_RANGE][TIME_RANGE_END_TIME] = end_date
    payload[FILTERS][FILTERS_TIME_ZONE] = "UTC"
    payload[QUERIES] = query_filters

    response = requests.post(QUERY_URL, json=payload, headers=POST_HEADERS)
    return response.json()



In [144]:
def parse_date(date_str: str):
    """Parse the given date"""
    date = datetime.datetime.fromisoformat(date_str).date()
    return date

In [145]:
def parse_performance_response(resp_json: dict):
    """Parse performance response and return a dataframe with values"""
    results = resp_json[DATA][RESULTS]
    data = {}

    df_skeleton = {
        DATE: [],
        VALUE: [],
    }
    for query_key, result in resp_json[DATA][RESULTS].items():
        metric = result[METRIC]
        values = result[DATA]
        skeleton = deepcopy(df_skeleton)
        for value in values:
            date = parse_date(value[0])
            value = value[1]

            skeleton[DATE].append(date)
            skeleton[VALUE].append(value)

        data[metric] = pd.DataFrame(skeleton)

    return data

In [146]:
all_metrics = get_model_metrics(PROJECT_NAME, MODEL_NAME)

250702T12:14:18.504Z     INFO| http: https://preprod.cloud.fiddler.ai/v3/projects GET -- emit req (0 B, timeout: (5, 100)) 
250702T12:14:18.792Z     INFO| http: https://preprod.cloud.fiddler.ai/v3/projects GET -- resp code: 200, took 0.281 s, resp/req body size: (434 B, 0 B) 
250702T12:14:18.796Z     INFO| http: https://preprod.cloud.fiddler.ai/v3/models GET -- emit req (0 B, timeout: (5, 100)) 
250702T12:14:19.907Z     INFO| http: https://preprod.cloud.fiddler.ai/v3/models GET -- resp code: 200, took 1.110 s, resp/req body size: (979 B, 0 B) 
250702T12:14:19.908Z     INFO| http: https://preprod.cloud.fiddler.ai/v3/models/431b97c7-1800-4448-b777-b3c7ba56c4a9 GET -- emit req (0 B, timeout: (5, 100)) 
250702T12:14:20.242Z     INFO| http: https://preprod.cloud.fiddler.ai/v3/models/431b97c7-1800-4448-b777-b3c7ba56c4a9 GET -- resp code: 200, took 0.334 s, resp/req body size: (5568 B, 0 B) 


In [147]:
with open('metrics_resp.json', 'w') as f:
    json.dump(all_metrics, f, indent=2)

In [148]:
extract_model_columns(all_metrics)

[ColumnInfo(id='__ANY__', name='All columns'),
 ColumnInfo(id='creditscore', name='creditscore'),
 ColumnInfo(id='geography', name='geography'),
 ColumnInfo(id='gender', name='gender'),
 ColumnInfo(id='age', name='age'),
 ColumnInfo(id='tenure', name='tenure'),
 ColumnInfo(id='balance', name='balance'),
 ColumnInfo(id='numofproducts', name='numofproducts'),
 ColumnInfo(id='hascrcard', name='hascrcard'),
 ColumnInfo(id='isactivemember', name='isactivemember'),
 ColumnInfo(id='estimatedsalary', name='estimatedsalary'),
 ColumnInfo(id='predicted_churn', name='predicted_churn'),
 ColumnInfo(id='churn', name='churn'),
 ColumnInfo(id='customer_id', name='customer_id'),
 ColumnInfo(id='timestamp', name='timestamp')]

In [149]:
perf_metrics = extract_performance_metrics(all_metrics)

In [150]:
perf_metrics[0].id

'auc'

In [151]:
model = get_model(PROJECT_NAME, MODEL_NAME)

250702T12:14:21.090Z     INFO| http: https://preprod.cloud.fiddler.ai/v3/projects GET -- emit req (0 B, timeout: (5, 100)) 
250702T12:14:21.363Z     INFO| http: https://preprod.cloud.fiddler.ai/v3/projects GET -- resp code: 200, took 0.272 s, resp/req body size: (434 B, 0 B) 
250702T12:14:21.365Z     INFO| http: https://preprod.cloud.fiddler.ai/v3/models GET -- emit req (0 B, timeout: (5, 100)) 
250702T12:14:21.842Z     INFO| http: https://preprod.cloud.fiddler.ai/v3/models GET -- resp code: 200, took 0.476 s, resp/req body size: (979 B, 0 B) 
250702T12:14:21.844Z     INFO| http: https://preprod.cloud.fiddler.ai/v3/models/431b97c7-1800-4448-b777-b3c7ba56c4a9 GET -- emit req (0 B, timeout: (5, 100)) 
250702T12:14:22.122Z     INFO| http: https://preprod.cloud.fiddler.ai/v3/models/431b97c7-1800-4448-b777-b3c7ba56c4a9 GET -- resp code: 200, took 0.276 s, resp/req body size: (5568 B, 0 B) 


In [159]:
model.project_id

UUID('0dde103e-bee5-4cbf-b28f-954b99cb6f32')

In [152]:
now = get_current_time()
days_ago = get_days_ago(7)

In [153]:
perf_details = query_performance_metrics(PROJECT_NAME, MODEL_NAME, now, days_ago)

250702T12:14:22.148Z     INFO| http: https://preprod.cloud.fiddler.ai/v3/projects GET -- emit req (0 B, timeout: (5, 100)) 
250702T12:14:22.432Z     INFO| http: https://preprod.cloud.fiddler.ai/v3/projects GET -- resp code: 200, took 0.283 s, resp/req body size: (434 B, 0 B) 
250702T12:14:22.435Z     INFO| http: https://preprod.cloud.fiddler.ai/v3/models GET -- emit req (0 B, timeout: (5, 100)) 
250702T12:14:22.937Z     INFO| http: https://preprod.cloud.fiddler.ai/v3/models GET -- resp code: 200, took 0.501 s, resp/req body size: (979 B, 0 B) 
250702T12:14:22.939Z     INFO| http: https://preprod.cloud.fiddler.ai/v3/models/431b97c7-1800-4448-b777-b3c7ba56c4a9 GET -- emit req (0 B, timeout: (5, 100)) 
250702T12:14:23.215Z     INFO| http: https://preprod.cloud.fiddler.ai/v3/models/431b97c7-1800-4448-b777-b3c7ba56c4a9 GET -- resp code: 200, took 0.275 s, resp/req body size: (5568 B, 0 B) 
250702T12:14:23.217Z     INFO| http: https://preprod.cloud.fiddler.ai/v3/projects GET -- emit req (0 B

In [154]:
perf_details

{'data': {'organization': {'id': 'febfb97f-6e51-4a43-a66c-86bab1c0c6fb',
   'name': 'preprod'},
  'project': {'id': '0dde103e-bee5-4cbf-b28f-954b99cb6f32', 'name': 'py_test'},
  'query_type': 'MONITORING',
  'time_comparison': None,
  'filters': {'time_label': '7d',
   'time_range': {'start_time': '2025-07-02T17:44:22',
    'end_time': '2025-06-25T17:44:22'},
   'time_zone': 'UTC',
   'bin_size': 'Day'},
  'results': {'a0a978b2-b990-4b59-abb6-f7ec34792bd5': {'col_names': ['timestamp',
     'auc'],
    'data': [['2025-06-26T00:00:00+00:00', 1.0],
     ['2025-06-27T00:00:00+00:00', 1.0],
     ['2025-06-28T00:00:00+00:00', 1.0],
     ['2025-06-29T00:00:00+00:00', 0.9353146853146853],
     ['2025-06-30T00:00:00+00:00', 0.9012345679012346],
     ['2025-07-01T00:00:00+00:00', 1.0],
     ['2025-07-02T00:00:00+00:00', None]],
    'query_key': 'a0a978b2-b990-4b59-abb6-f7ec34792bd5',
    'model': {'id': '431b97c7-1800-4448-b777-b3c7ba56c4a9',
     'name': 'bank_churn_simple_monitoring',
     've

In [160]:
results = parse_performance_response(perf_details)

In [162]:
results.keys()

dict_keys(['auc', 'auroc', 'accuracy', 'calibrated_threshold', 'expected_calibration_error', 'f1_score', 'fpr', 'geometric_mean', 'log_loss', 'precision', 'recall', 'data_count'])

In [163]:
results['fpr']

,date,value
0,2025-06-26,0.000000
1,2025-06-27,0.000000
2,2025-06-28,0.125000
3,2025-06-29,0.076923
4,2025-06-30,0.000000
5,2025-07-01,0.000000
6,2025-07-02,NaN
